In [175]:
from functions_v2 import*
from methods import MethodFinder

import brightway2 as bw
import bw2data as bd
import bw2analyzer as ba

#reduce?
import ast
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [176]:
bd.projects.set_current("premise-validation-try1")
bw.bw2setup()

bio3=bw.Database('biosphere3')
ei39=bw.Database('ecoinvent 3.9.1 cutoff')
ei39SSP2=bw.Database('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27')

Biosphere database already present!!! No setup is needed


LCA DATA TO EXCEL 

In [177]:
#sector filters file names/paths

cement = 'cement_small.yaml'
electricity = 'electricity_small.yaml'
fuels= 'fuels_small.yaml'
steel = 'steel_small.yaml'
transport = 'transport_small.yaml'

In [178]:
files_dict={}
files_dict['Cement']={'yaml': 'yamls\cement_small.yaml',
                      'yaml identifier': 'Cement'}
files_dict['Electricity']= {'yaml':'yamls\electricity_small.yaml',
                            'yaml identifier': 'Electricity'} #yaml identifier is the name of the filter in the yaml file, in the first line.
files_dict

{'Cement': {'yaml': 'yamls\\cement_small.yaml', 'yaml identifier': 'Cement'},
 'Electricity': {'yaml': 'yamls\\electricity_small.yaml',
  'yaml identifier': 'Electricity'}}

In [179]:
# Assuming generate_sets_from_filters is a predefined function
def process_yaml_files(files_dict, database):
    main_dict=files_dict.copy()
    for key, value in main_dict.items():
        yaml_file = value['yaml']
        yaml_identifier = value['yaml identifier']
        
        # Generate the sector activities
        sector_activities = generate_sets_from_filters(yaml_file, database=database)
        
        # Convert the set of activities to a list
        activities_list = list(sector_activities[yaml_identifier])
        
        # Add to the sectors_dict
        main_dict[key]['activities'] = activities_list
        
    return main_dict


In [ ]:
sectors_dict= process_yaml_files(files_dict, ei39SSP2)

In [ ]:
type(sectors_dict['Electricity']['activities'][0])

In [ ]:
sectors_dict['Electricity']['activities'][0].as_dict()['name']

In [ ]:
sectors_dict['Electricity']['activities'][4].as_dict()['name']

In [180]:
#Get Methods
finder=MethodFinder()
finder.find_and_create_method(criteria=['IPCC', '2013', 'GWP100'], exclude=['no LT'])
finder.find_and_create_method(criteria=['EN15804','Cumulative', 'non-renewable' ])
# finder.find_and_create_method(criteria=['land occupation','selected'])
# finder.find_and_create_method(criteria=['EN15804','fresh water'])
method_dict=finder.get_all_methods()
method_dict

{'method_1': {'object': Brightway2 Method: IPCC 2013: climate change: global warming potential (GWP100),
  'method name': ('IPCC 2013',
   'climate change',
   'global warming potential (GWP100)'),
  'short name': 'global warming potential (GWP100)',
  'unit': 'kg CO2-Eq'},
 'method_2': {'object': Brightway2 Method: EN15804: inventory indicators ISO21930: Cumulative Energy Demand - non-renewable energy resources,
  'method name': ('EN15804',
   'inventory indicators ISO21930',
   'Cumulative Energy Demand - non-renewable energy resources'),
  'short name': 'Cumulative Energy Demand - non-renewable energy resources',
  'unit': 'megajoule'}}

In [181]:
def sector_lca_scores(main_dict, method_dict):
    '''
    Generates the LCA scores for the sectors activities in the main dictionary 
    for the different methods in the method dictionary.

    It returns the main dictionary updated as scores dictionary which also holds the former information for each sector.
    The LCA scores are stored by method name in the respective sector dictionary within the main dictionary.
    '''

    # Initialize scores_dict as a copy of main_dict
    scores_dict = main_dict.copy()

    # Loop through each sector in main_dict
    for sector in scores_dict.keys():
        # Extract activities for the current sector
        sector_activities = scores_dict[sector]['activities']
        
        # Calculate LCA scores using the specified method
        lca_scores = compare_activities_multiple_methods(
            activities_list=sector_activities,
            methods=method_dict,
            identifier=sector,
            mode='absolute'
        )
        
        # Apply the small_inputs_to_other_column function with the cutoff value
        lca_scores = small_inputs_to_other_column(lca_scores, cutoff=0.02)
        
        # Save the LCA scores to the scores_dict
        scores_dict[sector]['lca_scores'] = lca_scores

    return scores_dict

In [182]:
scores_dict=sector_lca_scores(sectors_dict, method_dict) 

# this function is omitting common parts in the activities names, because only one type of activity portland and biomass from ICGGC is looked at 
# the results are 'Portland' and 'plant' in the activity column

Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'electricity production, at biomass-fired IGCC power '
Omitting activity name common prefix: 'electricity production, at biomass-fired IGCC power '


In [ ]:
scores_dict['Cement']['lca_scores'].keys()

In [ ]:
scores_dict['Electricity']['lca_scores'].keys()

In [ ]:
scores_dict['Cement'].keys()

In [183]:
def add_statistics(df, column_name='total'):

    #Need a rank row to plot the total LCA scores in descending order (satter opepyxl function takes in non categorial values)
    df['rank'] = df[column_name].rank(method="first", ascending="False")

    # Calculate mean, standard deviation, and IQR
    df['mean'] = df[column_name].mean()
    df['2std_abv'] = df['mean'] + df[column_name].std() * 2
    df['2std_blw'] = df['mean'] - df[column_name].std() * 2
    df['q1'] = df[column_name].quantile(0.25)
    df['q3'] = df[column_name].quantile(0.75)
    
    # Reorder the columns to place the new columns after 'total'
    cols = df.columns.tolist()
    total_index = cols.index(column_name) + 1
    new_cols = ['rank', 'mean', '2std_abv', '2std_blw', 'q1', 'q3']
    cols = cols[:total_index] + new_cols + cols[total_index:-len(new_cols)]
    
    return df[cols]


In [184]:
def sector_lca_scores_to_excel(scores_dict, excel_file_name):
    # Prepare to save each LCA score table to a different worksheet in the same Excel file
    excel_file = excel_file_name
    with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
        for sector in scores_dict.keys():
            lca_scores = scores_dict[sector]['lca_scores']
            for method, table in lca_scores.items():
                # Create a DataFrame for the current LCA score table
                df = pd.DataFrame(table)
                # Add the index as a new column at the beginning
                df.insert(0, 'Index', df.index)
                 # Add statistics to the DataFrame
                df = add_statistics(df)
                # Add sector marker column
                df['sector']=str(sector) # potentially remove!
                # Reorder the columns to move 'sector' after 'product'
                columns = list(df.columns)
                product_index = columns.index('product')
                # Insert 'sector' after 'product'
                columns.insert(product_index + 1, columns.pop(columns.index('sector')))
                # Reassign the DataFrame with the new column order
                df = df[columns]

                # Generate a worksheet name
                worksheet_name = f"{method}" #f"{sector}_{method}"
                if len(worksheet_name) > 31:
                    worksheet_name = worksheet_name[:31]
                # Save the DataFrame to the Excel file in a new worksheet
                df.to_excel(writer, sheet_name=worksheet_name, index=False)

In [205]:
lca_scores_in_excel=sector_lca_scores_to_excel(scores_dict, 'output_v51_2.xlsx')

PLOTS IN EXCEL

In [ ]:
# Input : one excel file which holds the sector workesheets
# create a dictionary of worksheet names which fullfill the criteria.
# iterate through the list to get the worksheets
# Grouping sector graphs in one worksheet - for worksheets with same sector create a chart each in a worksheet called sector_charts



In [ ]:
from openpyxl import load_workbook

def categorize_sheets_by_sector(file_path):
    # Load the workbook
    workbook = load_workbook(filename=file_path, read_only=True)
    
    # Initialize a dictionary to hold sectors and their corresponding sheet names
    worksheet_dict = {}
    
    # Iterate over all sheet names in the workbook
    for sheet_name in workbook.sheetnames:
        # Split the sheet name to extract the sector (assumes sector is the first part)
        sector = sheet_name.split('_')[0]
        
        # Add the sheet name to the corresponding sector in the dictionary
        if sector in worksheet_dict:
            worksheet_dict[sector].append(sheet_name)
        else:
            worksheet_dict[sector] = [sheet_name]
    
    return worksheet_dict

In [206]:
worksheet_dict=categorize_sheets_by_sector('output_v51_2.xlsx')

Function to plot categorical scatter plots (Note: Axis can't take on categorical labels)

In [186]:
from openpyxl.chart import ScatterChart, Reference, Series
from openpyxl import load_workbook
from openpyxl.drawing.text import CharacterProperties
from openpyxl.chart.label import DataLabelList


def dot_plots(filepath_workbook, worksheet_dict):
    
    # Load the workbook
    wb = load_workbook(filepath_workbook)
    
    # Iterate over each sector and its associated worksheets
    for sector, worksheet_names in worksheet_dict.items():
        
        # Create or get the chart sheet for the current sector
        chart_sheet_name = f"{sector}_dot_charts"
        if chart_sheet_name in wb.sheetnames:
            ws_charts = wb[chart_sheet_name]
        else:
            ws_charts = wb.create_sheet(chart_sheet_name)
                
        # Initial position for the first chart
        current_row = 1  # Start placing charts from row 1
        chart_height = 21  # Number of rows a chart occupies, adjust as needed
        
        # Iterate over each worksheet name in the current sector
        for worksheet_name in worksheet_names:
            ws = wb[worksheet_name]

            # Find min_row, max_row and max_column
            max_row = ws.max_row
            print(max_row)
            max_column = ws.max_column
            min_row=1

            #define columns
            total_col=9
            rank_col=10
            index_col=1
            mean_col=11
            std_adv_col=12
            std_blw_col=13
            q1_col=14
            q3_col=15
            
            # Create a ScatterChart (or other chart type as needed)
            chart = ScatterChart()

            # Chart titles
            chart.title = f"{ws['F2'].value} LCA scores for {sector} sector"
            chart.y_axis.title = f"{ws['G2'].value}"
            chart.x_axis.title = 'activity rank'
            # Avoid overlap
            chart.title.overlay = False
            chart.x_axis.title.overlay = False
            chart.y_axis.title.overlay = False 


            # Define the data range for the chart
            y_values = Reference(ws, min_col=total_col, min_row=min_row, max_row=max_row)
            print(y_values)
            x_values = Reference(ws, min_col=rank_col, min_row=min_row, max_row=max_row)
            print(x_values)

            # Create a series and add it to the chart
            series = Series(y_values, x_values, title_from_data=True)
            chart.series.append(series)
            chart.style = 9

            # Customize the series to show only markers (dots)
            series.marker.symbol = "circle"
            series.marker.size = 5
            series.graphicalProperties.line.noFill = True


            # ADJUST X-AXIS
            chart.x_axis.tickLblPos = "low"
            chart.x_axis.majorGridlines = None
            chart.x_axis.tickMarkSkip = 0  # Show all tick marks
            chart.x_axis.tickLblSkip = 0  # Show all labels

            chart.x_axis.scaling.orientation = "minMax"
            chart.x_axis.crosses = "autoZero"
            chart.x_axis.axPos = "b"
            chart.x_axis.delete = False

            # ADJUST Y-AXIS

            # Calculate the min and max y-values to determine the axis range
            y_values_list = [ws.cell(row=row, column=total_col).value for row in range(2,max_row+1)] #number of rows need to be defined based on dataframe
            min_y_value = min(y_values_list)-min(y_values_list)*0.1
            max_y_value = max(y_values_list)+ max(y_values_list)*0.1

            # Set y-axis range with some padding
            padding = (max_y_value - min_y_value) * 0.1  # 10% padding on each side
            chart.y_axis.scaling.min = min_y_value - padding
            chart.y_axis.scaling.max = max_y_value + padding

            # Explicitly set tick labels to be visible
            chart.y_axis.tickLblPos = "nextTo"  # Position the labels next to the tick marks
            #chart.y_axis.majorGridlines = True  # Ensure major gridlines are enabled (though you mentioned they're visible)
            chart.y_axis.delete = False  # Ensure axis is not deleted
            # Format the y-axis to show 2 decimal places
            chart.y_axis.number_format = '0.000'


            # ---------
            # ADD STATS
            # ---------
            # MEAN
            # Add a new series to the chart for the mean line
            mean_y = Reference(ws, min_col=mean_col, min_row=min_row, max_row=max_row)
            mean_series = Series(mean_y, x_values, title_from_data="True")
            chart.series.append(mean_series)

            # Customize the mean series to show as a line
            mean_series.marker.symbol = "none"  # No markers, just a line
            mean_series.graphicalProperties.line.solidFill = "FF0000"  # Red line for mean value
            mean_series.graphicalProperties.line.width = 10000  # Set line width (default units are EMUs)

            # IQR
            # Add a new series to the chart for the iqr lines
            iqr1= Reference(ws, min_col=q1_col, min_row=min_row, max_row=max_row)
            iqr3= Reference(ws, min_col=q3_col, min_row=min_row, max_row=max_row)
            iqr1_series = Series(iqr1, x_values, title_from_data="True")
            iqr3_series = Series(iqr3, x_values, title_from_data="True")
            chart.series.append(iqr1_series)
            chart.series.append(iqr3_series)

            # Customize the iqr 1,3 series to show as a line
            iqr1_series.marker.symbol = "none"  # No markers, just a line
            iqr3_series.marker.symbol = "none"
            iqr1_series.graphicalProperties.line.solidFill = "6082B6"  # Blue line 
            iqr3_series.graphicalProperties.line.solidFill = "6082B6"  
            iqr1_series.graphicalProperties.line.width = 10000  # Set line width (default units are EMUs)
            iqr3_series.graphicalProperties.line.width = 10000  # Set line width (default units are EMUs)

            # STD
            # Add a new series to the chart for the std lines
            std_abv= Reference(ws, min_col=std_adv_col, min_row=min_row, max_row=max_row)
            std_blw= Reference(ws, min_col=std_blw_col, min_row=min_row, max_row=max_row)
            std_abv_series = Series(std_abv, x_values, title_from_data="True")
            std_blw_series = Series(std_blw, x_values, title_from_data="True")
            chart.series.append(std_abv_series)
            chart.series.append(std_blw_series)

            # Customize the iqr 1,3 series to show as a line
            std_abv_series.marker.symbol = "none"  # No markers, just a line
            std_blw_series.marker.symbol = "none"
            std_abv_series.graphicalProperties.line.solidFill = "FFC300"  # yellow line
            std_blw_series.graphicalProperties.line.solidFill = "FFC300"  
            std_abv_series.graphicalProperties.line.width = 10000  # Set line width (default units are EMUs)
            std_blw_series.graphicalProperties.line.width = 10000  # Set line width (default units are EMUs)

            # Set legend position to the right of the plot area
            chart.legend.position = 'r'  # 'r' for right
            chart.legend.overlay= False

            # Adjust chart dimensions and position to make space for the legend
            chart.width = 18  # Example width, adjust as needed
            chart.height = 10  # Example height, adjust as needed

            # Add the chart to the chart worksheet
            # Calculate the position for this chart
            position = f"A{current_row}"
            ws_charts.add_chart(chart, position)
            
            # Update current_row to position the next chart below the current one
            current_row += chart_height

    wb.save(filepath_workbook)

In [190]:
worksheet_dict

{'Cement': ['Cement_global_warming_potential',
  'Cement_cumulative_energy_demand'],
 'Electricity': ['Electricity_global_warming_pote',
  'Electricity_cumulative_energy_d']}

In [207]:
dot_plots('output_v51_2.xlsx', worksheet_dict=worksheet_dict)

8
'Cement_global_warming_potential'!$I$1:$I$8
'Cement_global_warming_potential'!$J$1:$J$8
8
'Cement_cumulative_energy_demand'!$I$1:$I$8
'Cement_cumulative_energy_demand'!$J$1:$J$8
11
'Electricity_global_warming_pote'!$I$1:$I$11
'Electricity_global_warming_pote'!$J$1:$J$11
11
'Electricity_cumulative_energy_d'!$I$1:$I$11
'Electricity_cumulative_energy_d'!$J$1:$J$11


Setup Stacked Plots

In [ ]:
from openpyxl import Workbook
from openpyxl.chart import BarChart, Series, Reference
from openpyxl.chart.text import RichText
from openpyxl.drawing.text import CharacterProperties

def stacked_bars(filepath_workbook, worksheet_dict):

    # Load the workbook
    wb = load_workbook(filepath_workbook)
    
    # Iterate over each sector and its associated worksheets
    for sector, worksheet_names in worksheet_dict.items():
        
        # Create or get the chart sheet for the current sector
        chart_sheet_name = f"{sector}_stacked_bars"
        if chart_sheet_name in wb.sheetnames:
            ws_charts = wb[chart_sheet_name]
        else:
            ws_charts = wb.create_sheet(chart_sheet_name)
                
        # Initial position for the first chart
        current_row = 1  # Start placing charts from row 1
        chart_height = 21  # Number of rows a chart occupies, adjust as needed
        
        # Iterate over each worksheet name in the current sector
        for worksheet_name in worksheet_names:
            ws = wb[worksheet_name]

            # Find min_row, max_row and max_column
            max_row = ws.max_row
            max_column = ws.max_column
            input_min_col=15

            chart = BarChart()
            chart.type = "col"
            chart.style = 2
            chart.grouping = "stacked"
            chart.overlap = 100

            # Chart titles
            chart.title = f"{ws['F2'].value} input contributions to LCA scores for {sector} sector"
            chart.y_axis.title = f"{ws['G2'].value}"
            chart.x_axis.title = 'activity index'

            # Avoid overlap
            chart.title.overlay = False
            chart.x_axis.title.overlay = False
            chart.y_axis.title.overlay = False 
            chart.legend.overlay = False

            # define data
            data = Reference(ws, min_col=input_min_col, min_row=1, max_row=max_row, max_col = max_column)
            cats = Reference(ws, min_col=1, min_row=2, max_row=max_row)

            chart.add_data(data, titles_from_data=True)
            chart.set_categories(cats)
            chart.shape = 4

            # Modify each series in the chart to disable the inversion of negative values 
            for series in chart.series:
                series.invertIfNegative = False

            # y-axis ticks
            chart.y_axis.tickLblPos = "nextTo"
            chart.y_axis.delete = False  # Ensure axis is not deleted
            chart.y_axis.number_format = '0.000'

            # Set legend position to the right of the plot area
            chart.legend.position = 'r'  # 'r' for right
            chart.legend.overlay= False

            # Adjust chart dimensions and position to make space for the legend
            chart.width = 15  # Example width, adjust as needed
            chart.height = 25  # Example height, adjust as needed

            # Add the chart to the chart worksheet
            # Calculate the position for this chart
            position = f"A{current_row}"
            ws_charts.add_chart(chart, position)
            
            # Update current_row to position the next chart below the current one
            current_row += chart_height+4

    wb.save(filepath_workbook)


In [ ]:
# stacked_bars('output_v4_2.xlsx', worksheet_dict=worksheet_dict)

In [202]:
from openpyxl import load_workbook
from openpyxl.chart import BarChart, Reference

def stacked_bars(filepath_workbook, worksheet_dict):
    # Load the workbook
    wb = load_workbook(filepath_workbook)
    
    # Iterate over each sector and its associated worksheets
    for sector, worksheet_names in worksheet_dict.items():
        
        # Create or get the chart sheet for the current sector
        chart_sheet_name = f"{sector}_stacked_bars"
        if chart_sheet_name in wb.sheetnames:
            ws_charts = wb[chart_sheet_name]
        else:
            ws_charts = wb.create_sheet(chart_sheet_name)
                
        # Initial position for the first chart
        current_row = 1  # Start placing charts from row 1
        current_col = 1  # Start placing charts from column 1
        chart_height = 18  # Number of rows a chart occupies
        chart_width = 15   # Number of columns a chart occupies
        charts_per_row = 3  # Number of charts per row
        
        # Iterate over each worksheet name in the current sector
        for i, worksheet_name in enumerate(worksheet_names):
            ws = wb[worksheet_name]

            # Find min_row, max_row and max_column
            max_row = ws.max_row
            max_column = ws.max_column
            input_min_col = 16

            chart = BarChart()
            chart.type = "col"
            chart.style = 2
            chart.grouping = "stacked"
            chart.overlap = 100

            # Chart titles
            chart.title = f"{ws['F2'].value} input contributions to LCA scores for {sector} sector"
            chart.y_axis.title = f"{ws['G2'].value}"
            chart.x_axis.title = 'activity index'

            # Avoid overlap
            chart.title.overlay = False
            chart.x_axis.title.overlay = False
            chart.y_axis.title.overlay = False 
            chart.legend.overlay = False

            # Define data
            data = Reference(ws, min_col=input_min_col, min_row=1, max_row=max_row, max_col=max_column)
            cats = Reference(ws, min_col=1, min_row=2, max_row=max_row)

            chart.add_data(data, titles_from_data=True)
            chart.set_categories(cats)
            chart.shape = 4

            # Modify each series in the chart to disable the inversion of negative values 
            for series in chart.series:
                series.invertIfNegative = False

            # y-axis ticks
            chart.y_axis.tickLblPos = "nextTo"
            chart.y_axis.delete = False  # Ensure axis is not deleted
            chart.y_axis.number_format = '0.000'

            # Set legend position to the right of the plot area
            chart.legend.position = 'r'  # 'r' for right
            chart.legend.overlay = False

            # Adjust chart dimensions
            chart.width = 15  # Example width, adjust as needed
            chart.height = 25  # Example height, adjust as needed

            # Add the chart to the chart worksheet
            # Calculate the position for this chart
            position = ws_charts.cell(row=current_row, column=current_col).coordinate
            ws_charts.add_chart(chart, position)
            
            # Update position for the next chart
            current_col += chart_width -4
            if (i + 1) % charts_per_row == 0:  # Move to the next row after placing `charts_per_row` charts
                current_row += chart_height +2
                current_col = 1  # Reset to the first column
        
    wb.save(filepath_workbook)


In [208]:
stacked_bars('output_v51_2.xlsx', worksheet_dict=worksheet_dict)

In [ ]:
from openpyxl import Workbook
from openpyxl.chart import BarChart, Series, Reference
from openpyxl.chart.text import RichText
from openpyxl.drawing.text import CharacterProperties

wb = load_workbook('output_v45_2.xlsx')
ws = wb['Electricity_Electricity_cumulat']
ws_charts=wb.create_sheet('stacked_bar_charts_cumulat_10')

chart = BarChart()
chart.type = "col"
chart.style = 2
chart.grouping = "stacked"
chart.overlap = 100

chart.title = "Stacked bar chart"
chart.y_axis.title = "method unit"
chart.x_axis.title = "activity indexes"

# Avoid overlap
chart.title.overlay = False
chart.x_axis.title.overlay = False
chart.y_axis.title.overlay = False 
chart.legend.overlay = False

# define data
data = Reference(ws, min_col=15, min_row=1, max_row=11, max_col = 19)
cats = Reference(ws, min_col=1, min_row=2, max_row=11)

chart.add_data(data, titles_from_data=True)
chart.set_categories(cats)
chart.shape = 7 #what is this for?

# Modify each series in the chart to disable the inversion of negative values 
for series in chart.series:
    series.invertIfNegative = False

# y-axis ticks
chart.y_axis.tickLblPos = "nextTo"
chart.y_axis.delete = False  # Ensure axis is not deleted
chart.y_axis.number_format = '0.000'

ws_charts.add_chart(chart, "B1")
wb.save('output_v47_2.xlsx')
